## Artificial Neural Networks (ANN)

## Building Neural Nets from Scratch

We will build neural networks based upon the basic principles briefly presented. The intention is to create a very simple model and understand how it works.

**Please note that this code is not optimized and not to be used in production**. 

This is for instructive purpose - for us to understand how ANN works. 

Libraries like `TensorFlow` have highly optimized code.

(*The following code is inspired from [these](https://github.com/dennybritz/nn-from-scratch) terrific notebooks*)

In [ ]:
# Import the required packages
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import scipy

In [ ]:
# Display plots inline 
%matplotlib inline
# Define plot's default figure size
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

In [ ]:
import random
random.seed(123)

In [ ]:
# Read the datasets
train = pd.read_csv("data/intro_to_ann.csv")

# Run the following if running with Colab
#train = pd.read_csv("https://github.com/fordanic/cmiv-ai-course/raw/master/notebooks/data/intro_to_ann.csv")

In [ ]:
X, y = np.array(train.iloc[:,0:2]), np.array(train.iloc[:,2])

In [ ]:
# Let's plot the dataset and see what it looks like
plt.scatter(X[:,0], X[:,1], s=40, c=y)

## Our ANN 

We will here define all the building blocks needed to build a simple neural network.

In [ ]:
# Cretae NxM matrix with random values between [a,b]
def initMatrix(N, M, a=0, b=0):
    return np.random.rand(N , M)*(b - a) + a

# A sigmoid activation function
def sigmoid(x):
    return 1/(1+np.exp(-x))

# The derivative of our activation/sigmoid function, in terms of the output (i.e. y)
def dsigmoid(y):
    return y - y**2


class ANN:
    def __init__(self, ni, nh, no):
        # Number of input, hidden, and output nodes
        self.ni = ni + 1 # +1 for bias node
        self.nh = nh
        self.no = no

        # Activations for nodes
        self.ai = np.ones((self.ni,))
        self.ah = np.ones((self.nh,))
        self.ao = np.ones((self.no,))
        
        # Create weights
        self.wi = initMatrix(self.ni, self.nh, -0.2, 0.2)
        self.wo = initMatrix(self.nh, self.no, -2.0, 2.0)
                
        # last change in weights for momentum   
        self.ci = initMatrix(self.ni, self.nh)        
        self.co = initMatrix(self.nh, self.no)
                

    def backPropagate(self, targets, N, M):
        
        if len(targets) != self.no:
            print(targets)
            raise ValueError('wrong number of target values')

        # Calculate error terms for output                
        error = targets-self.ao
        output_deltas = dsigmoid(self.ao) * error
        
        # output_deltas = np.zeros(self.no)
        # for k in range(self.no):
        #    error = targets[k]-self.ao[k]
        #    output_deltas[k] = dsigmoid(self.ao[k]) * error

        
        # Calculate error terms for hidden        
        error = np.dot(self.wo, output_deltas)
        hidden_deltas = dsigmoid(self.ah) * error
                
        #hidden_deltas = np.zeros(self.nh)
        #for j in range(self.nh):
        #    error = 0.0
        #    for k in range(self.no):
        #        error += output_deltas[k]*self.wo[j][k]
        #    hidden_deltas[j] = dsigmoid(self.ah[j]) * error

                
        # Update output weights
        change = np.outer(self.ah, output_deltas)
        self.wo += N*change + M*self.co
        self.co = change
        
        #for j in range(self.nh):
        #    for k in range(self.no):
        #        change = output_deltas[k] * self.ah[j]
        #        self.wo[j][k] += N*change + M*self.co[j][k]
        #        self.co[j][k] = change

                
        # Update input weights
        change = np.outer(self.ai, hidden_deltas)
        self.wi += N*change + M*self.ci
        self.ci = change
        
        #for i in range(self.ni):
        #    for j in range(self.nh):
        #        change = hidden_deltas[j]*self.ai[i]
        #        self.wi[i][j] += N*change + M*self.ci[i][j]
        #        self.ci[i][j] = change

                
        # Calculate error
        error = 0.5*np.sum((targets - self.ao)**2)
        
        #error = 0.0
        #for k in range(len(targets)):
        #    error += 0.5*(targets[k]-self.ao[k])**2
        return error


    def test(self, patterns):
        self.predict = np.empty([len(patterns), self.no])
        for i, p in enumerate(patterns):
            self.predict[i] = self.activate(p)            
            
    def activate(self, inputs):
        
        if len(inputs) != self.ni-1:
            print(inputs)
            raise ValueError('wrong number of inputs')

              
        # Input activations
        self.ai[:-1] = inputs
        
        # for i in range(self.ni-1):
        #     self.ai[i] = inputs[i]
        
        
        # Hidden activations
        self.ah = sigmoid(np.dot(self.ai.T, self.wi))        
        
        # for j in range(self.nh):
        #     sum_h = 0.0
        #    for i in range(self.ni):
        #        sum_h += self.ai[i] * self.wi[i][j]
        #    self.ah[j] = sigmoid(sum_h)
        
        
        # Output activations
        self.ao = sigmoid(np.dot(self.ah.T, self.wo))
                                            
        # for k in range(self.no):
        #   sum_o = 0.0
        #    for j in range(self.nh):
        #        sum_o += self.ah[j] * self.wo[j][k]
        #    self.ao[k] = sigmoid(sum_o)

        return self.ao[:]
    

    def train(self, patterns, iterations=1000, N=0.5, M=0.1):
        # N: learning rate
        # M: momentum factor
        patterns = list(patterns)
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.activate(inputs)
                error += self.backPropagate([targets], N, M)
            if i % 5 == 0:
                print('error in interation %d : %-.5f' % (i,error))
            print('Final training error: %-.5f' % error)

### Running the model on our dataset

In [ ]:
# Create a network with two inputs, five hidden, and one output nodes
ann = ANN(2, 30, 1)

%timeit -n 1 -r 1 ann.train(zip(X,y), iterations=90)

### Let's visualize and observe the results

In [ ]:
# Helper function to plot a decision boundary.
# This generates the contour plot to show the decision boundary visually
def plot_decision_boundary(nn_model):
    # Set min and max values and give it some padding
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), 
                         np.arange(y_min, y_max, h))
    # Predict the function value for the whole gid
    nn_model.test(np.c_[xx.ravel(), yy.ravel()])
    Z = nn_model.predict
    Z[Z>=0.5] = 1
    Z[Z<0.5] = 0
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.scatter(X[:, 0], X[:, 1], s=40,  c=y)

In [ ]:
plot_decision_boundary(ann)
plt.title("Our initial model")

___
## Exercise

Create Neural networks with 10 hidden nodes on the above code. 

What's the impact on accuracy?

In [ ]:
ann = ANN(2, 50, 1)
%timeit -n 1 -r 1 ann.train(zip(X,y), iterations=50)
plot_decision_boundary(ann)
plt.title("Our next model with 10 hidden units")

___

___
## Exercise

Train the neural networks by increasing the epochs. 

What's the impact on accuracy?

In [ ]:
ann = ANN(2, 10, 1)
%timeit -n 1 -r 1 ann.train(zip(X,y), iterations=100)
plot_decision_boundary(ann)
plt.title("Our model with 10 hidden units and 100 iterations")

___